In [1]:
import socket
from pathlib import Path


class Pathfinder:
    def __init__(self, topdirname=None):
        IDE = Path.cwd()
        if not self.on_lxplus and topdirname is not None:
            while IDE.name != topdirname:
                IDE = IDE.parent
            self.IDE = IDE.resolve()
        elif self.on_lxplus:
            print("Running on lxplus I see...")
            self.IDE = Path.cwd()
        else:
            self.IDE = Path.cwd()

        print("Current working directory:")
        print(self.IDE)
        print("Used files' paths' templates:")
        for key, value in self.templates.items():
            print(f"{key}: {value.__code__.co_varnames}")

    @property
    def on_lxplus(self) -> bool:
        # Check hostname
        hostname = socket.gethostname()

        # Check for LXPLUS-specific directories
        lxplus_paths = [
            Path("/afs/cern.ch").exists(),
            Path("/eos").exists(),
        ]

        # Determine environment based on hostname and paths
        is_lxplus = "lxplus" in hostname.lower() or any(lxplus_paths)

        return is_lxplus

    @property
    def templates(self) -> dict:
        templates = {
            "Bs2DsPi/MC/2015/Up": lambda lparent: lparent
            / f"Bs2DsPi/MC/Stripping24r1_Sim09c/B2DX_MC_13264021_Bs_Dspi_KKpi_2015_up_Bs_BDTG_KKPi_withETA.root",
            "Bs2DsPi/MC/2015/Down": lambda lparent: lparent
            / f"Bs2DsPi/MC/Stripping24r1_Sim09c/B2DX_MC_13264021_Bs_Dspi_KKpi_2015_dw_Bs_BDTG_KKPi_withETA.root",
            "Bs2DsPi/MC/2016/Up": lambda lparent: lparent
            / f"Bs2DsPi/MC/Stripping28r1_Sim09h/B2DX_MC_13264021_Bs_Dspi_KKpi_2016_up_Bs_BDTG_KKPi_withETA.root",
            "Bs2DsPi/MC/2016/Down": lambda lparent: lparent
            / f"Bs2DsPi/MC/Stripping28r1_Sim09h/B2DX_MC_13264021_Bs_Dspi_KKpi_2016_dw_Bs_BDTG_KKPi_withETA.root",
            "Bs2DsPi/MC/2017/Up": lambda lparent: lparent
            / f"Bs2DsPi/MC/Stripping29r2_Sim09h/B2DX_MC_13264021_Bs_Dspi_KKpi_2017_up_Bs_BDTG_KKPi_withETA.root",
            "Bs2DsPi/MC/2017/Down": lambda lparent: lparent
            / f"Bs2DsPi/MC/Stripping29r2_Sim09h/B2DX_MC_13264021_Bs_Dspi_KKpi_2017_dw_Bs_BDTG_KKPi_withETA.root",
            "Bs2DsPi/MC/2018/Up": lambda lparent: lparent
            / f"Bs2DsPi/MC/Stripping34_Sim09h/B2DX_MC_11164052_Bd_Ds-pi+_KKpi_2018_up_Bs_BDTG_KKPi_withETA.root",
            "Bs2DsPi/MC/2018/Down": lambda lparent: lparent
            / f"Bs2DsPi/MC/Stripping34_Sim09h/B2DX_MC_11164052_Bd_Ds-pi+_KKpi_2018_dw_Bs_BDTG_KKPi_withETA.root",
        }
        return templates

    # @staticmethod
    # def determine_notation(decay):
    #     match decay:
    #         case "Bs2DsPi/Data":
    #             return 0
    #         case "Bs2DsPi/MC":
    #             return 99
    #         case "Bs2DsstPi/Data":
    #             return 1
    #         case "Bs2DsstPi/MC":
    #             return 99
    #         case _:
    #             raise ValueError(f"Unknown decay: {decay}. Please check the decay name.")

    def find_path(self, decay: str, year: int, mode: str):
        if self.on_lxplus:
            parent = Path("/eos/lhcb/wg/b2oc/BR_Bs2DsstPi_Run12/")
        else:
            parent = self.IDE / "data/Beauty/2025/"
        try:
            template = self.templates[f"{decay}/{year}/{mode}"](parent)
        except KeyError:
            template = None

        return template


# >>> Functions
def check_files(*files):
    missingFiles = []
    for ifile in files:
        if not ifile.exists():
            missingFiles.append(ifile)
    return missingFiles


def RCut(*cuts) -> str:
    return " && ".join(icut for icut in cuts)


# <<< Functions

In [2]:
# Needs variables: $IDE and $check_files()

# CONSTANTs
LUMINOSITIES = {2011: 435.0, 2012: 737.0, 2015: 998.0, 2016: 1160.0, 2017: 1178.0, 2018: 1178.0}
lumiSum = sum(LUMINOSITIES.values())
lumiFracFor = {iyear: LUMINOSITIES[iyear] / lumiSum for iyear in LUMINOSITIES.keys()}

YEARS = (
    # 2011,
    # 2012,  # overlap with 2011
    2015,
    2016,  # overlap with 2015
    2017,
    2018,
)  # Data from selected years (proper idx is determined by class Pathfinder's method)
MODES = ("Up", "Down")  # Up and Down modes for the MC samples
TREE_NAME = "DecayTree"
BF_BS2DSPI = 0.0001

In [3]:
ph = Pathfinder("the-respected-daimyo")

decays = {
    "Selected": "Bs2DsstPi/MC",
    "Normalization": "Bs2DsPi/MC",
}  # Decays to be used in the analysis tagged by "Selected" or "Normalization"

filesFor = {}
metadataFrom = {}
for idecay in decays.keys():
    filesFor[idecay] = {}  # Sorting data by a decay
    for jyear in YEARS:
        filesFor[idecay][jyear] = []  # Sorting data by a year
        for kmode in MODES:
            file_path = ph.find_path(decays[idecay], jyear, kmode)
            if file_path is None:
                continue
            elif not file_path.exists():
                print(f"File not found: {file_path}")
                continue
            filesFor[idecay][jyear].append(file_path)  # Sorting data by a mode (up or down)
        if len(filesFor[idecay][jyear]) == 0:
            filesFor[idecay].pop(jyear)  # Remove empty years

Running on lxplus I see...
Current working directory:
/eos/home-i01/j/jbreczew/Bs2DsPi-Analysis
Used files' paths' templates:
Bs2DsPi/MC/2015/Up: ('lparent',)
Bs2DsPi/MC/2015/Down: ('lparent',)
Bs2DsPi/MC/2016/Up: ('lparent',)
Bs2DsPi/MC/2016/Down: ('lparent',)
Bs2DsPi/MC/2017/Up: ('lparent',)
Bs2DsPi/MC/2017/Down: ('lparent',)
Bs2DsPi/MC/2018/Up: ('lparent',)
Bs2DsPi/MC/2018/Down: ('lparent',)


In [4]:
kinematic_cuts = {}
kinematic_cuts["Bs2DsPi/Data"] = {
    # Eta cuts
    "lab0_eta": "lab0_ETA > 2.0 && lab0_ETA < 5.0",
    # Eta for final state particles
    "lab1_eta": "lab1_ETA > 2.0 && lab1_ETA < 5.0",
    "lab3_eta": "lab3_ETA > 2.0 && lab3_ETA < 5.0",
    "lab4_eta": "lab4_ETA > 2.0 && lab4_ETA < 5.0",
    "lab5_eta": "lab5_ETA > 2.0 && lab5_ETA < 5.0",
    # RICH information
    "lab1_rich": "lab1_hasRich == 1",
    "lab3_rich": "lab3_hasRich == 1",
    "lab4_rich": "lab4_hasRich == 1",
    "lab5_rich": "lab5_hasRich == 1",
    # Number of tracks
    "n_tracks": "nTracks >= 15 && nTracks <= 500",
    # # Final state particle momentum  # GIVES ZERO COUNT
    # "lab1_p": "lab1_P >= 2 && lab1_P <= 150",
    # "lab3_p": "lab3_P >= 2 && lab3_P <= 150",
    # "lab4_p": "lab4_P >= 2 && lab4_P <= 150",
    # "lab5_p": "lab5_P >= 2 && lab5_P <= 150",
    # # Final state particle transverse momentum  # GIVES ZERO COUNT
    # "lab1_pt": "lab1_PT >= 0.25 && lab1_PT <= 45",
    # "lab3_pt": "lab3_PT >= 0.25 && lab3_PT <= 45",
    # "lab4_pt": "lab4_PT >= 0.25 && lab4_PT <= 45",
    # "lab5_pt": "lab5_PT >= 0.25 && lab5_PT <= 45",
    # Semileptonic cut
    "lab1_muon": "lab1_isMuon == 0",
    "lab3_muon": "lab3_isMuon == 0",
    "lab4_muon": "lab4_isMuon == 0",
    "lab5_muon": "lab5_isMuon == 0",
    # Charm meson flight distance
    "charm_fd": "lab2_FD_ORIVX > 0",
    # Charm meson FD chi^2
    "charm_fdchi2": "lab2_FDCHI2_ORIVX > 2",
    # Charm meson lifetime
    "charm_lifetime": "lab0_LifetimeFit_Dplus_ctau[0] > 0",
    # # Bs invariant mass  # GIVES ERROR
    # "bs_mass": "lab0_MassFitConsD_M >= 5000 && lab0_MassFitConsD_M <= 7000",
    # Charm invariant mass
    "charm_mass": "lab2_MM >= 1948 && lab2_MM <= 1988",
    # BDT response
    "bdt_response": "BDTGResponse_3 > 0.4",
}

kinematic_cuts["Bs2DsstPi/Data"] = {
    # Eta cuts
    "lab0_eta": "lab0_ETA > 2.0 && lab0_ETA < 5.0",
    # Eta for final state particles
    "lab1_eta": "lab1_ETA > 2.0 && lab1_ETA < 5.0",
    "lab3_eta": "lab3_ETA > 2.0 && lab3_ETA < 5.0",
    "lab4_eta": "lab4_ETA > 2.0 && lab4_ETA < 5.0",
    "lab5_eta": "lab5_ETA > 2.0 && lab5_ETA < 5.0",
    # RICH information
    "lab1_rich": "lab1_hasRich == 1",
    "lab3_rich": "lab3_hasRich == 1",
    "lab4_rich": "lab4_hasRich == 1",
    "lab5_rich": "lab5_hasRich == 1",
    # Number of tracks
    "n_tracks": "nTracks >= 15 && nTracks <= 500",
    # Final state particle momentum
    "lab1_p": "lab1_P >= 2 && lab1_P <= 150",
    "lab3_p": "lab3_P >= 2 && lab3_P <= 150",
    "lab4_p": "lab4_P >= 2 && lab4_P <= 150",
    "lab5_p": "lab5_P >= 2 && lab5_P <= 150",
    # Final state particle transverse momentum
    "lab1_pt": "lab1_PT >= 0.25 && lab1_PT <= 45",
    "lab3_pt": "lab3_PT >= 0.25 && lab3_PT <= 45",
    "lab4_pt": "lab4_PT >= 0.25 && lab4_PT <= 45",
    "lab5_pt": "lab5_PT >= 0.25 && lab5_PT <= 45",
    # Semileptonic cut
    "lab1_muon": "lab1_isMuon == 0",
    "lab3_muon": "lab3_isMuon == 0",
    "lab4_muon": "lab4_isMuon == 0",
    "lab5_muon": "lab5_isMuon == 0",
    # Charm meson flight distance
    "charm_fd": "lab2_FD_ORIVX > 0",
    # Charm meson FD chi^2
    "charm_fdchi2": "lab2_FDCHI2_ORIVX > 2",
    # Charm meson lifetime
    "charm_lifetime": "lab0_LifetimeFit_Dplus_ctau[0] > 0",
    # Bs invariant mass
    "bs_mass": "lab0_MassFitConsD_M >= 5000 && lab0_MassFitConsD_M <= 7000",
    # Charm invariant mass
    "charm_mass": "lab2_MM >= 1948 && lab2_MM <= 1988",
    # BDT response
    "bdt_response": "BDTGResponse_3 > 0.4",
}
kinematic_cuts["Bs2DsPi/MC"] = kinematic_cuts["Bs2DsPi/Data"].copy()
kinematic_cuts["Bs2DsstPi/MC"] = kinematic_cuts["Bs2DsstPi/Data"].copy()

In [5]:
# Needs variables: $files, $decays, $BFFor and any of cut variables

from ROOT import RDataFrame as RDF  # type: ignore
import ROOT as RT  # type: ignore
from tqdm import tqdm

RT.ROOT.EnableImplicitMT()  # type: ignore
RT.gROOT.SetBatch(True)  # type: ignore


In [11]:
NFor = {}
NCutedFor = {}
εFor = {}

for iyear in YEARS:
    print(f"{iyear}")
    for jdecay in decays.keys():
        if filesFor[jdecay] == {}:
            continue
        print(f"| {jdecay}")
        cuts = RCut(*list(kinematic_cuts[decays[jdecay]].values()))
        print("| | Cuts", cuts)
        print("|")
        NFor[jdecay] = {}
        for kyear, kmodes in filesFor[jdecay].items():
            if kyear == iyear:
                print("| |",TREE_NAME, end=": ")
                print("| |","N of files:",len([str(lfile) for lfile in kmodes]))
                data = RDF(TREE_NAME, [str(lfile) for lfile in kmodes])
                dataCuted = data.Filter(cuts)
                print("| |","Before Cut Count:",data.Count().GetValue())
                print("| |","After Cut Count:",dataCuted.Count().GetValue())
                print()


# Main Calculation Loop (multi RDF)
# for iyear in tqdm(YEARS.keys()):
#     for jdecay in decays.keys():
#         data[jdecay] = RDF(TREE_NAME, [str(ifile) for ifile in filesFor[jdecay]])
#         dataCuted[jdecay] = data[jdecay].Filter(cuts)
#         NFor[jdecay] = data[jdecay].Count().GetValue()
#         NCutedFor[jdecay] = dataCuted[jdecay].Count().GetValue()
#         εFor[jdecay] = NFor[jdecay] / NCutedFor[jdecay]

# BranFracs[decays["Selected"]] = (NFor["Selected"]/NFor["Normalization"])*(εFor["Normalization"]/εFor["Selected"])*BranFracs[decays["Normalization"]]

2015
| Normalization
| | Cuts lab0_ETA > 2.0 && lab0_ETA < 5.0 && lab1_ETA > 2.0 && lab1_ETA < 5.0 && lab3_ETA > 2.0 && lab3_ETA < 5.0 && lab4_ETA > 2.0 && lab4_ETA < 5.0 && lab5_ETA > 2.0 && lab5_ETA < 5.0 && lab1_hasRich == 1 && lab3_hasRich == 1 && lab4_hasRich == 1 && lab5_hasRich == 1 && nTracks >= 15 && nTracks <= 500 && lab1_isMuon == 0 && lab3_isMuon == 0 && lab4_isMuon == 0 && lab5_isMuon == 0 && lab2_FD_ORIVX > 0 && lab2_FDCHI2_ORIVX > 2 && lab0_LifetimeFit_Dplus_ctau[0] > 0 && lab2_MM >= 1948 && lab2_MM <= 1988 && BDTGResponse_3 > 0.4
|
| | DecayTree: | | N of files: 2
| | Before Cut Count: 363295
| | After Cut Count: 189326

2016
| Normalization
| | Cuts lab0_ETA > 2.0 && lab0_ETA < 5.0 && lab1_ETA > 2.0 && lab1_ETA < 5.0 && lab3_ETA > 2.0 && lab3_ETA < 5.0 && lab4_ETA > 2.0 && lab4_ETA < 5.0 && lab5_ETA > 2.0 && lab5_ETA < 5.0 && lab1_hasRich == 1 && lab3_hasRich == 1 && lab4_hasRich == 1 && lab5_hasRich == 1 && nTracks >= 15 && nTracks <= 500 && lab1_isMuon == 0 && lab3_i